In [2]:
"""
1.CROP标签需要删除
2.除了B、I标签，还有结束E标签、单个字符实体S标签需要添加上
3./usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
"""
import os
import json
import random

In [ ]:
def replace_text_in_file(file_path, old_text, new_text):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    content = content.replace(old_text, new_text)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_files_in_directory(directory, old_text, new_text):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt') or file.endswith('.json'):
                file_path = os.path.join(root, file)
                replace_text_in_file(file_path, old_text, new_text)


# 使用示例
directory_path = 'G:/python_codes/Principles_of_Computer_Construction_KG/corpus'  # 替换为你的目录路径
old_text = 'CROP'
new_text = 'CORP'
process_files_in_directory(directory_path, old_text, new_text)

In [11]:
def remove_unlabled_shots_from_corpus(corpus: dict, blank_seq: list):
    for n in blank_seq:
        for idx in range(n * 8, (n + 1) * 8):
            try:
                del corpus[str(idx)]
            except:
                pass
    return corpus

def remove_null_dict_from_labels(corpus_labels: dict, blank_seq: list):
    for n in blank_seq:
        del corpus_labels[n]
    return corpus_labels

def initial_dataset(corpus: dict, corpus_labels: dict):
    dataset_dict = {"corpus": [], "labels": []}
    for key, value in corpus.items():
        dataset_dict['corpus'].append(value)
    for key, value in corpus_labels.items():
        for i in value:
            dataset_dict['labels'].append(i)

    return dataset_dict

In [12]:
def process_labels(dataset_dict: dict):
    dataset_dict['new_labels'] = []
    for i, sentence in enumerate(dataset_dict['corpus']):
        tmp = ['O'] * len(sentence)
        labels = dataset_dict['labels'][i]['output']
        for label in labels:
            for idx in range(label['start'], label['end']):
                tmp[idx] = f'I-{label['type']}'
        dataset_dict['new_labels'].append(tmp)

    return dataset_dict

In [13]:
with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\corpus\texts_sentence.json", "r", encoding="utf-8") as f:
    corpus = json.loads(f.read())

with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\corpus\corpus_labels.txt", "r", encoding="utf-8") as f:
    corpus_labels = eval(f.read())

In [14]:
blank_seq = []
for k, v in corpus_labels.items():
    if not v:
        blank_seq.append(k)

In [15]:
len(corpus)

5653

In [16]:
new_corpus = remove_unlabled_shots_from_corpus(corpus, blank_seq)
new_corpus_labels = remove_null_dict_from_labels(corpus_labels, blank_seq)

In [17]:
dataset_dict = initial_dataset(new_corpus, new_corpus_labels)

In [18]:
dataset_dict['corpus'][0]

'计算机系统不同于一般的电子设备，它是一个由硬件、软件组成的复杂的自动化设备'

In [19]:
dataset_dict['labels'][0]['output']

[{'type': 'PROD', 'start': 0, 'end': 5, 'prob': 0.12411527, 'span': '计算机系统'}]

In [20]:
len(dataset_dict['corpus']) == len(dataset_dict['labels'])

True

- 训练集：验证集：测试集 = 8：1：1
- 先随机从数据集中取出1000条数据
- 对这1000条数据进行校对（设立一个新的标签PCC。没有识别出来的要标记出来，改正识别错的）
- 校对完之后，建立一个词典
- 用这1000条数据进行预训练
- 再拿出1000条数据用第一次训练后的模型进行标注，再进行一次人工标注后，更新词典。到此为止完成第一次迭代，下面重复这个过程，直到训练集的数据全部标注完成

In [21]:
trainset_slice = int(len(dataset_dict['corpus']) * 0.8)
test_set_sclice = int(len(dataset_dict['corpus']) * 0.1)
validset_slice = len(dataset_dict['corpus']) - trainset_slice - test_set_sclice

In [27]:
trainset_slice

4147

In [39]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset = json.loads(f.read())
all_idx = list(range(trainset_slice))
first_epoch_idx = first_epoch_trainset['ids']
second_epoch_idx = list(
    random.sample([i for i in all_idx if i not in first_epoch_idx], 1000)
)
third_epoch_idx = list(
    random.sample([i for i in all_idx if i not in (first_epoch_idx + second_epoch_idx)], 1000)
)
forth_epoch_idx = list(
    random.sample([i for i in all_idx if i not in (first_epoch_idx + second_epoch_idx + third_epoch_idx)], 1000)
)
fifth_epoch_idx = list(set(all_idx) - set(first_epoch_idx) - set(second_epoch_idx) - set(third_epoch_idx) - set(forth_epoch_idx))

In [42]:
def generate_epoch_trainset(idx, dataset_dict, epoch='first'):
    epoch_trainset = {
        'ids': idx,
        "corpus": [dataset_dict["corpus"][i] for i in idx],
        "labels": [dataset_dict["labels"][i] for i in idx],
    }
    with open(f'G:/python_codes/Principles_of_Computer_Construction_KG/output/{epoch}_epoch_trainset.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(epoch_trainset))

In [46]:
generate_epoch_trainset(second_epoch_idx, dataset_dict, epoch='second')
generate_epoch_trainset(third_epoch_idx, dataset_dict, epoch='third')
generate_epoch_trainset(forth_epoch_idx, dataset_dict, epoch='forth')
generate_epoch_trainset(fifth_epoch_idx, dataset_dict, epoch='fifth')

In [41]:
(len(fifth_epoch_idx) + len(forth_epoch_idx) + len(third_epoch_idx) + len(second_epoch_idx) + len(first_epoch_idx)) == len(all_idx)

True

In [47]:
test_set = {
        'corpus': dataset_dict['corpus'][trainset_slice:trainset_slice + test_set_sclice],
        'labels': dataset_dict['labels'][trainset_slice:trainset_slice + test_set_sclice],
    }
validset = {
        'corpus': dataset_dict['corpus'][trainset_slice + test_set_sclice:],
        'labels': dataset_dict['labels'][trainset_slice + test_set_sclice:],
    }

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(test_set))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(validset))

In [ ]:
first_epoch_trainset_idx = list(
    random.sample(range(0, trainset_slice), 1000)
)
first_epoch_trainset = {
    'ids': first_epoch_trainset_idx,
    "corpus": [dataset_dict["corpus"][i] for i in first_epoch_trainset_idx],
    "labels": [dataset_dict["labels"][i] for i in first_epoch_trainset_idx],
}
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(first_epoch_trainset))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v2.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset = json.loads(f.read())

In [48]:
json_template = {
    "data": {"text": None},
    "predictions": [
        {
            "model_version": "nlp_raner_named-entity-recognition_chinese-large-generic",
            "score": 0.94,
            "result": [],
        }
    ],
}
result_template = {
        'id': None,
        'from_name': 'label',
        'to_name': 'text',
        "type": "labels",
        "value": {
              "start": None,
              "end": None,
              "score": None,  # prob
              "text": None,  # span
              "labels": [
                None  # type
              ]
            }
    }

In [2]:
def generate_pre_labeled_dataset(dataset: dict):
    pre_labeled_data = []
    id_count = 1

    for i, labels in enumerate(dataset["labels"]):

        json_template_tmp = {
            "data": {"text": None},
            "predictions": [
                {
                    "model_version": "nlp_raner_named-entity-recognition_chinese-large-generic",
                    "score": 0.86,
                    "result": [],
                }
            ],
        }

        json_template_tmp["data"]["text"] = dataset["corpus"][i]

        for label in labels["output"]:

            json_template_tmp["predictions"][0]["result"].append(
                {
                    "id": str(id_count),
                    "from_name": "label",
                    "to_name": "text",
                    "type": "labels",
                    "value": {
                        "start": label["start"],
                        "end": label["end"],
                        "score": label["prob"],  # prob
                        "text": label["span"],  # span
                        "labels": [label["type"]],  # type
                    },
                }
            )
            id_count += 1

        pre_labeled_data.append(json_template_tmp)

    return pre_labeled_data

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_v2_pre_labeled_data.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(first_epoch_trainset)))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_test_set.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(test_set)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_validset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(validset)))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v2.json', 'r', encoding='utf-8') as f:
    test_set_v2 = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v2.json', 'r', encoding='utf-8') as f:
    validset_v2 = json.loads(f.read())

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_test_set_v2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(test_set_v2)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\pre_labeled_validset_v2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(generate_pre_labeled_dataset(validset_v2)))

In [ ]:
entity_set = set()
for i, labels in enumerate(first_epoch_trainset['labels']):
    entitys = [label['span'] for label in labels['output']]
    entitys_type = [label['type'] for label in labels['output']]
    for item in dict(zip(entitys, entitys_type)).items():
        entity_set.add(item)

In [ ]:
entity_list = list(entity_set)
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(entity_list))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.txt', 'w', encoding='utf-8') as f:
    f.write(str(entity_list))

In [ ]:
new_dataset_dict = process_labels(dataset_dict)

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\datasets_backup.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(dataset_dict))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\datasets.json', 'r', encoding='utf-8') as f:
    test = json.loads(f.read())